# 基于大模型的文本内容智能评判BaseLineV1.0

赛题背景：
大模型作为人工智能领域的热点技术，已在众多垂直行业中展现出巨大的应用潜能。然而，随着其应用范围的不断拓宽，针对大模型的评测需求也日趋复杂化、个性化，这一趋势对现有的评判技术提出了严峻挑战。
在文本生成领域，由于信息的多样性、主观性，以及评价标准的复杂性，传统自动化评估方法效果较差，灵活性不足，而人工评价方式效率低下，成本高昂，难以满足当前大规模的评判需求。因此，如何运用自动化、智能化的手段，实现对文本内容的高效评判，成为了业界亟待解决的问题。
在此背景下，本赛题以“基于大模型的文本内容智能评判”作为主题，旨在借助大模型强大的语义理解能力和泛化能力，应对不同领域和场景的评判需求，同时精准对齐人类专家的评判标准，进一步提升评判的准确性和可靠性。

赛题任务：
本赛题主要考察文本自动评判任务，参赛者需要利用大模型技术，构建一个智能评判模型，同时具备客观题目、创作题目的评判能力。其中，客观题目需结合参考答案作出评判，创作题目需对齐人类专家的主观评分，评判维度涉及流畅性、规范性。

In [1]:
# 封装对象
class DataObject:
    def __init__(self, data_id, creation_requirements, content, evaluation_dimension, score):
        self.data_id = data_id  # 数据编号
        self.creation_requirements = creation_requirements  # 创作要求（仅规范性数据包含）/选择题题目
        self.content = content  # 待评判内容/选择题答案及模型回复
        self.evaluation_dimension = evaluation_dimension  # 评判维度
        self.score = score  # 分值

    def to_string(self):
        return f"""
<data_id>{self.data_id}</data_id>
<creation_requirements>{self.creation_requirements}</creation_requirements>
<content>{self.content}</content>
<evaluation_dimension>{self.evaluation_dimension}</evaluation_dimension>
<score>{self.score}</score>
"""

# 示例用法
data_object = DataObject(
    data_id=1,
    creation_requirements="请描述选择题的题目",
    content="待评判内容/选择题答案及模型回复",
    evaluation_dimension="评判维度",
    score=5
)

# 将对象转换为字符串
result_string = data_object.to_string()
print(result_string)


<data_id>1</data_id>
<creation_requirements>请描述选择题的题目</creation_requirements>
<content>待评判内容/选择题答案及模型回复</content>
<evaluation_dimension>评判维度</evaluation_dimension>
<score>5</score>



In [16]:
import pandas as pd

class DataObject:
    def __init__(self, data_id, creation_requirements, content, evaluation_dimension, score):
        self.data_id = data_id  # 数据编号
        self.creation_requirements = creation_requirements  # 创作要求（仅规范性数据包含）/选择题题目
        self.content = content  # 待评判内容/选择题答案及模型回复
        self.evaluation_dimension = evaluation_dimension  # 评判维度
        self.score = score  # 分值

    def to_string(self):
        return f"""
        <evaluation_dimension>{self.evaluation_dimension}</evaluation_dimension>
        <creation_requirements>{self.creation_requirements}</creation_requirements>
        <content>{self.content}</content>

"""

# 读取CSV文件并存储到列表中
def read_csv_to_data_objects(file_path):
    # 读取CSV文件，跳过第一行
    df = pd.read_csv(file_path, encoding="utf-8",skiprows=1, header=None, names=[
                     'data_id', 'creation_requirements', 'content', 'evaluation_dimension', 'score'])

    # 创建DataObject实例的列表
    data_objects = [
        DataObject(row['data_id'], row['creation_requirements'], row['content'],
                   row['evaluation_dimension'], row['score'])
        for _, row in df.iterrows()
    ]

    return data_objects

# 示例用法
file_path = 'train_e.csv'
data_objects = read_csv_to_data_objects(file_path)
print(f"len = {len(data_objects)}")
# 打印每个对象的字符串表示
for i in range(2):
    print(data_objects[i].to_string())


len = 3500

        <evaluation_dimension>美国塔科马市（47°17'N，122°28'W，是北太平洋东岸的港口城市，位于，人口约21.9万（2020年）。该市一位名叫约克的年轻人，非常喜爱中国文化的，工作之余经常前往图书馆读书，或漫步公园游憩。1873年，横贯美洲大陆的北太平洋铁路建成。这条铁路成了分处铁路两端的约克曾祖父母缔结姻缘的纽带。他们初识时，塔科马位于铁路西端，千余人仅居住于此，到1889年人口达3.6万。图12示意塔科马市内部空间结构。</evaluation_dimension>
        <creation_requirements>nan</creation_requirements>
        <content>流畅性</content>



        <evaluation_dimension>中国人民银行上海总部，各省、自治区、直辖市及计划单列市分行；各政策性银行、国有商业银行，中国邮政储蓄银行，各股份制商业银行：
中国人民银行位于上海的总部机构，以及遍布全国各省、自治区、直辖市以及计划单列市的分行；同时，各政策性银行、所有国有商业银行、中国邮政储蓄银行，以及各股份制商业银行均被告知：</evaluation_dimension>
        <creation_requirements>请以“针对中国人民银行及其各级分支机构，以及各类银行机构（包括政策性银行、国有商业银行、中国邮政储蓄银行和股份制商业银行）发布的一项通知或指示，但由于文章正文部分未给出，具体通知或指示的内容无法确定。因此，总结为：该文章是对上述金融机构发出的一个正式通知或指示，但具体内容和目的需要查看文章的详细内容才能得知。中国人民银行决定自2024年5月18日起下调个人住房公积金贷款利率，其中首套和二套房的贷款利率均有所降低。”为主题写一篇文章</creation_requirements>
        <content>规范性</content>




In [1]:
# 构造思维链
cot_prompt = f"""
请根据以下步骤对user提供的文本进行评分。请按照每一步提供的指导进行操作。

1. 识别题型：
    - 阅读 <evaluation_dimension></evaluation_dimension> 标签中的内容。
    - 判断该内容是“选择题”还是“非选择题”。

2. 如果是选择题：
    - 检查 <content></content> 中的答案是否正确。
    - 如果答案正确，输出 score=1。
    - 如果答案错误，输出 score=0。

3. 如果是非选择题：
    - 阅读 <evaluation_dimension></evaluation_dimension> 标签中的内容。
    - 判断该维度是流畅性还是规范性。

4. 如果维度是流畅性：
    - 先阅读 <creation_requirements></creation_requirements>，再阅读 <content></content>。
    - 根据不同的评分标准，具体步骤如下：
    - 根据以下标准对文章的语句通顺性、语法错误、字词错误、表述方式进行评分：
        - 1分：非常不流畅，不具备可读性。语法错误明显，难以理解；大量拼写错误和错别字，影响阅读；表达不清晰，难以捉摸要表达的意思。(2.5,∞]个错误/百字。
        - 2分：具有可读性，但较不流畅。常见语法错误多，需花费一定时间理解；一些拼写错误和错别字，阅读中断；表达较为模糊，需用一些猜测才能明白含义。(2,2.5]个错误/百字。
        - 3分：基本流畅，存在少量语法错误，但影响较小。稍有拼写错误，但不影响阅读；主要意思表达清楚，但部分地方表述不够准确。(1,2]个错误/百字。
        - 4分：较流畅，语法错误稀少，易读性较高。几乎无拼写错误，阅读顺畅；表达清晰、准确，容易理解。(0.5,1]个错误/百字。
        - 5分：非常流畅，语法、拼写完美，阅读体验优秀。表达精炼、准确、得体；文句优美，行文连贯，思维严密。[0,0.5]个错误/百字。

5. 如果维度是规范性：
    - 先阅读 <creation_requirements></creation_requirements>，再阅读 <content></content>。
    - 根据以下标准对文章的内容遵循性、格式规范性、语句逻辑、层次结构进行评分：
        - 1分：创作内容离题，与提示语句要求不符，格式非常不规范。杂乱无章，句子结构混乱，缺乏逻辑。(5,∞]个错误/千字。
        - 2分：创作内容与提示语句要求有一定契合但覆盖不全，格式较不规范。缺乏清晰的结构，但基本逻辑仍能找到。(4,5]个错误/千字。
        - 3分：创作内容与提示语句要求基本契合但覆盖不全，格式一般规范。结构基本顺畅，逻辑较清晰。(2,4]个错误/千字。
        - 4分：创作内容与提示语句要求基本契合且基本覆盖，格式较规范。结构清晰，逻辑条理分明。(1,2]个错误/千字。
        - 5分：创作内容与提示语句要求完美契合，格式非常规范。结构严谨，逻辑清晰，层次分明。[0,1]个错误/千字。

6. 最终输出：
    - 根据以上分析，给出最终的评分，并以“score=分数”的形式输出。
"""

import re





In [2]:
import pandas as pd

class DataObject:
    def __init__(self, data_id, creation_requirements, content, evaluation_dimension, score=None):
        self.data_id = data_id  # 数据编号
        self.creation_requirements = creation_requirements  # 创作要求（仅规范性数据包含）/选择题题目
        self.content = content  # 待评判内容/选择题答案及模型回复
        self.evaluation_dimension = evaluation_dimension  # 评判维度
        self.score = score  # 分值

    def to_string(self):
        return f"""
        <evaluation_dimension>{self.evaluation_dimension}</evaluation_dimension>
        <creation_requirements>{self.creation_requirements}</creation_requirements>
        <content>{self.content}</content>

"""

# 读取CSV文件并存储到列表中
def read_csv_to_data_objects(file_path):
    # 读取CSV文件，跳过第一行
    df = pd.read_csv(file_path, encoding="utf-8",skiprows=1, header=None, names=[
                     'data_id', 'creation_requirements', 'evaluation_dimension', 'content'])

    # 创建DataObject实例的列表
    data_objects = [
        DataObject(row['data_id'], row['creation_requirements'], row['content'],
                   row['evaluation_dimension'])
        for _, row in df.iterrows()
    ]

    return data_objects

# 示例用法
file_path = 'test_e.csv'
data_objects = read_csv_to_data_objects(file_path)

print(data_objects[333].evaluation_dimension)



规范性


In [3]:
import time
import asyncio
import json
import os
import nest_asyncio
from tqdm.asyncio import tqdm_asyncio
from openai import AsyncOpenAI
import re
import csv

# 允许在已有事件循环中嵌套调用异步代码
nest_asyncio.apply()
async def fetch_completion(client, data_object, model_name, n):
    messages = [
        {"role": "system", "content": cot_prompt},
        {"role": "user", "content": data_object.to_string()}
    ]
    start_time = time.time()
    completion = await client.chat.completions.create(
        model=model_name,
        messages=messages
    )
    elapsed_time = time.time() - start_time
    response = completion.choices[0].message.content
    print(f"第 {n+1} 条数据用时: {elapsed_time:.2f} 秒")
    # print(f"第 {n+1} 条数据输出为: {response}")
    # 使用正则表达式匹配score后的值
    pattern = r"score=([0-9])"
    # 搜索匹配项
    match = re.search(pattern, response)
    if match:
        score = int(match.group(1))
    else:
        score = 0
    
    return {
        "data_id": data_object.data_id,
        "evaluation_dimension":data_object.evaluation_dimension,
        "score":score
    }

# async def baseline_model(client, data_objects, model_name):
#     res = []
#     data_objects_list = [fetch_completion(client, data_objects[i], model_name, i) for i in range(len(data_objects))]
#     for result in tqdm_asyncio.as_completed(data_objects_list, desc='Processing data_objects'):
#         result_data = await result
#         # print(f'Processing data_object: {result_data["data_id"]}')
#         res.append(result_data)
#     return res


async def baseline_model(client, data_objects, model_name):
    res = []
    batch_size = max(1, len(data_objects) // 15)  # Calculate batch size as 1/10 of data_objects
    for i in range(0, len(data_objects), batch_size):
        batch = data_objects[i:i + batch_size]
        tasks = [fetch_completion(client, obj, model_name, i + j) for j, obj in enumerate(batch)]
        for result in tqdm_asyncio.as_completed(tasks, desc='Processing data_objects'):
            result_data = await result
            res.append(result_data)
    return res
    
def write_results_to_csv(results, file_path):
    os.makedirs(file_path, exist_ok=True)
    submit_path = os.path.join(file_path, "submit.csv")
    
    # CSV header
    headers = ["数据编号", "评判维度", "预测分数"]
    
    # Write results to CSV file
    with open(submit_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        
        # Write the header
        writer.writerow(headers)
        
        # Write each result as a row
        for result in results:
            writer.writerow([result["data_id"], result["evaluation_dimension"], result["score"]])


async def main():
    
    client = AsyncOpenAI(api_key='sk-wjkljhfdhajfhkadfkh',
                         base_url='http://0.0.0.0:23333/v1')
    model_cards = await client.models.list()._get_page()
    model_name = model_cards.data[0].id
    res = await baseline_model(client, data_objects, model_name)
    return res


In [4]:
# 运行异步主函数
# 使用示例
# results = await baseline_model(client, data_objects, model_name)
# write_results_to_csv(results, "path/to/save/")

all_start_time = time.time()
res = asyncio.run(main())
all_time = time.time() - all_start_time
write_results_to_csv(res, "./")
print(all_time)

Processing data_objects:   0%|          | 1/203 [00:18<1:02:11, 18.47s/it]

第 163 条数据用时: 18.29 秒


Processing data_objects:   1%|          | 2/203 [00:27<43:02, 12.85s/it]  

第 111 条数据用时: 26.96 秒


Processing data_objects:   1%|▏         | 3/203 [00:30<27:16,  8.18s/it]

...

第 3048 条数据用时: 3.02 秒


Processing data_objects:  75%|███████▌  | 3/4 [00:03<00:01,  1.08s/it]

第 3047 条数据用时: 3.78 秒


Processing data_objects: 100%|██████████| 4/4 [00:10<00:00,  2.59s/it]

第 3046 条数据用时: 10.34 秒
1921.0049304962158


In [5]:
print(res[0])

{'data_id': 'TE0163', 'evaluation_dimension': '规范性', 'score': 4}
